In [2]:
import random
from typing import Optional, Union, Dict, List

In [3]:

class DummyLLM:
    """
    A dummy LLM implementation that returns random responses
    """

    def __init__(self, temperature: float = 0.7):
        self.temperature = temperature
        self._responses = [
            "I think we should use the calculator tool for this task.",
            "Let me search through the available tools.",
            "Based on my analysis, we should proceed step by step.",
            "I recommend using the following approach...",
            "The solution requires mathematical computation.",
            "Let's break this problem down into smaller parts.",
        ]

    def generate(
        self,
        prompt: str,
        system_message: Optional[str] = None,
        temperature: Optional[float] = None,
    ) -> Dict[str, Union[str, float]]:
        """
        Generate a random response with metadata

        Args:
            prompt (str): The input prompt
            system_message (Optional[str]): System message to guide generation
            temperature (Optional[float]): Override default temperature

        Returns:
            Dict containing response text and metadata
        """
        # Use random confidence score between 0.3 and 0.9
        confidence = random.uniform(0.3, 0.9)

        return {
            "text": random.choice(self._responses),
            "confidence": confidence,
            "tokens_used": random.randint(10, 50),
            "finish_reason": "stop",
        }

    def batch_generate(
        self,
        prompts: List[str],
        system_message: Optional[str] = None,
        temperature: Optional[float] = None,
    ) -> List[Dict[str, Union[str, float]]]:
        """
        Generate multiple random responses

        Args:
            prompts (List[str]): List of input prompts
            system_message (Optional[str]): System message to guide generation
            temperature (Optional[float]): Override default temperature

        Returns:
            List of response dictionaries
        """
        return [
            self.generate(prompt, system_message, temperature) for prompt in prompts
        ]

In [4]:
# Test cases for DummyLLM class
def test_dummy_llm_generate():
    llm = DummyLLM()
    response = llm.generate("Test prompt")
    assert "text" in response
    assert "confidence" in response
    assert "tokens_used" in response
    assert "finish_reason" in response
    assert 0.3 <= response["confidence"] <= 0.9
    assert 10 <= response["tokens_used"] <= 50

def test_dummy_llm_batch_generate():
    llm = DummyLLM()
    prompts = ["Prompt 1", "Prompt 2"]
    responses = llm.batch_generate(prompts)
    assert len(responses) == len(prompts)
    for response in responses:
        assert "text" in response
        assert "confidence" in response
        assert "tokens_used" in response
        assert "finish_reason" in response
        assert 0.3 <= response["confidence"] <= 0.9
        assert 10 <= response["tokens_used"] <= 50


In [5]:
# Test cases for AgentMemory class
def test_agent_memory_add_step():
    memory = AgentMemory()
    step = {"action": "test_action", "result": "test_result"}
    memory.add_step(step)
    assert len(memory.steps) == 1
    assert memory.steps[0] == step

def test_agent_memory_reset():
    memory = AgentMemory()
    step = {"action": "test_action", "result": "test_result"}
    memory.add_step(step)
    memory.reset()
    assert len(memory.steps) == 0
    assert len(memory.conversation) == 0


In [6]:
# Test cases for BaseAgent class
class TestTool:
    def __init__(self, name):
        self.name = name

    def execute(self, input):
        return f"Executed {input}"

class TestAgent(BaseAgent):
    def generate_action(self, task: str) -> dict:
        return {"tool": "test_tool", "input": task}

    def execute_action(self, action: dict) -> Any:
        tool = self.tools[action["tool"]]
        return tool.execute(action["input"])

def test_base_agent_run():
    tools = [TestTool("test_tool")]
    model = lambda x: x  # Dummy model for demonstration
    agent = TestAgent(tools=tools, model=model)
    task = "Test task"
    result = agent.run(task)
    assert result == "Executed Test task"

def test_base_agent_update_memory():
    tools = [TestTool("test_tool")]
    model = lambda x: x  # Dummy model for demonstration
    agent = TestAgent(tools=tools, model=model)
    action = {"tool": "test_tool", "input": "Test input"}
    result = "Test result"
    agent.update_memory(action, result)
    assert len(agent.memory.steps) == 1
    assert agent.memory.steps[0]["action"] == action
    assert agent.memory.steps[0]["result"] == result
